In [1]:
import torch
import torchvision
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.datasets import MNIST # 데이터 셋셋from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) # 셋 테스트 목적

In [2]:
nn

<module 'torch.nn' from '/usr/local/lib/python3.7/dist-packages/torch/nn/__init__.py'>

In [3]:
class Generator(nn.Module):
    def __init__(self, z_dim=10, im_chan=1, hidden_dim=64):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        #Build the neural network
        self.gen = nn.Sequential(
            self.gen_block(z_dim, hidden_dim * 4),
            self.gen_block(hidden_dim * 4, hidden_dim*2, kernel_size=4, stride=1),
            self.gen_block(hidden_dim * 2, hidden_dim),
            self.gen_block(hidden_dim, im_chan, kernel_size=4, final_layer=True),
            
        )
    def gen_block(self, in_channel, out_channel, kernel_size= 3, stride=2,padding=0, final_layer = False):
      if not final_layer:
        return nn.Sequential(
            nn.ConvTranspose2d(in_channel, out_channel, kernel_size=kernel_size, stride = stride),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
        )
      else:
        return  nn.Sequential(
            nn.ConvTranspose2d(in_channel, out_channel, kernel_size, stride),#, padding=3
            nn.Tanh(),
        )
    
    def unsqueeze_noise(self, noise):
        return noise.view(len(noise), self.z_dim, 1, 1)
        
    def forward(self, noise):
        x = self.unsqueeze_noise(noise)
        return self.gen(x)
    
    

In [4]:
class Discriminator(nn.Module):
    def __init__(self, im_chan=1, hidden_dim=16):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            self.disc_block(im_chan, hidden_dim),
            self.disc_block(hidden_dim, hidden_dim*2),
            self.disc_block(hidden_dim*2, 1, final_layer=True),
        )

    def disc_block(self, in_channel, out_channel, kernel_size= 3, stride=2, final_layer = False):
      if not final_layer:
        return nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size, stride),
            nn.BatchNorm2d(out_channel),
            nn.LeakyReLU(0.2, inplace=True),
        )
      else: # Final Layer
        return  nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size, stride),
        )
        
    def forward(self, image):
      disc_pred = self.disc(image)
      return disc_pred.view(len(disc_pred),-1)

In [5]:
def get_noise(n_samples, z_dim, device='cuda'):
    return torch.randn(n_samples, z_dim, device=device)

In [6]:
criterion = nn.BCEWithLogitsLoss()
n_epochs= 50
z_dim = 10
display_step = 500
batch_size = 128
lr = 0.0002 #만약 디스크리미네이터는 감소하고 제네레이터 로스가 증가한다면 이 값을 줄이는 방법. 스케일 10배씩 줄여가도 됨.

beta_1 = 0.5
beta_2 = 0.999
device = 'cuda'

5-2 데이터 로딩

In [7]:
transform = transforms.Compose ([
              transforms.ToTensor(),
              transforms.Normalize((0.5,), (0.5,)),
            ])
dataloader = DataLoader(
  MNIST('.', download=True, transform=transform),
  batch_size = batch_size,
  shuffle = True)

In [8]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta_1, beta_2))
disc = Discriminator().to(device)
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr, betas=(beta_1, beta_2))

def weights_init(m):
  if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
    torch.nn.init.normal_(m.weight, 0.0, 0.02) #수정1 0.2-> 0.02
  if isinstance(m, nn.BatchNorm2d):
    torch.nn.init.normal_(m.weight, 0.0, 0.02) #수정2 0.2-> 0.02
    torch.nn.init.constant_(m.bias, 0)

gen = gen.apply(weights_init)
disc = disc.apply(weights_init)

In [9]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1,2,0).squeeze())
    #plt.imshow( image_grid.permute(1,2,0).squeeze()* 255)
    plt.show()

In [10]:
n_epochs = 50
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss =0
for epoch in range(n_epochs):
  for real, _ in tqdm(dataloader):
    cur_batch_size = len(real)
    real = real.to(device)

    #업데이트 디스크
    disc_opt.zero_grad()
    latent = get_noise (cur_batch_size, z_dim, device = device) # z

    fake = gen (latent) # G(z)
    disc_fake_pred = disc(fake.detach()) # D(G(z))
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred)) # log(1-D(G(z)))
    disc_real_pred = disc(real) # D(x)
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred)) # log(D(x))
    disc_loss = (disc_fake_loss + disc_real_loss) / 2

    # track the average disc loss
    mean_discriminator_loss += disc_loss.item() / display_step
    disc_loss.backward(retain_graph=True)
    disc_opt.step()

    #Update gen
    gen_opt.zero_grad()
    latent_2 = get_noise (cur_batch_size, z_dim, device=device) # z
    fake_2 = gen(latent_2) # G(z)
    disc_fake_pred = disc(fake_2) # D(G(z))
    gen_loss = criterion (disc_fake_pred, torch.ones_like(disc_fake_pred))
    gen_loss.backward()
    gen_opt.step()

    #Track the average gen loss
    mean_generator_loss += gen_loss.item() / display_step
    
    #print(fake.size())
    #print(fake)
    #print(fake.type())
    # visualize the results
    if cur_step % display_step == 0 and cur_step > 0:
          print(f"step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
          show_tensor_images(fake)#, size=(1,34,34)
          show_tensor_images(real)
          mean_generator_loss = 0
          mean_discriminator_loss = 0
    cur_step += 1

Output hidden; open in https://colab.research.google.com to view.